<a href="https://colab.research.google.com/github/LiChinChen1/LLM-pratice/blob/main/R3_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第三次作業

## Basline
將第一次作業的 QA 檢索聊天機器人，重新透過 LangChain 進行實作

### 載入必要套件

In [1]:
%%capture
!pip install python-dotenv
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install sentence_transformers
!pip install langchain
!pip install langchain-community
!pip install pypdf
!pip install yt_dlp
!pip install openai

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
from langchain.document_loaders import WebBaseLoader

import pypdf
import yt_dlp
import openai
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

#### 1.1 PDFs

In [3]:
from langchain.document_loaders import PyPDFLoader

fiel_dir = "https://www2.deloitte.com/content/dam/Deloitte/us/Documents/consulting/us-state-of-gen-ai-report-q2.pdf"


loader = PyPDFLoader(fiel_dir)
pages = loader.load()

#### 1.2 Youtube

In [4]:
!pip install pydub
import pydub

In [5]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

url = "https://www.youtube.com/watch?v=jGwO_UgTS7I"
save_dir = "docs/youtube/"


loader = GenericLoader(
    YoutubeAudioLoader([url], save_dir),
    OpenAIWhisperParser())
# video_docs = loader.load()
# video_docs[0].page_content[0:500]

#### 1.3 網路文章

In [6]:
from langchain.document_loaders import WebBaseLoader

url = "https://docms.gov.taipei/News_Content.aspx?n=3E256D29704D8C68&sms=658ECC19F2127F5C&s=934503982809E9DC"


loader = WebBaseLoader(url)
news_docs = loader.load()
news_docs[0].page_content[:100]

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n\t臺北市政府兵役局-服務區塊-役男入營前座談會常見問題彙總\r\n\n\n\n\n\n\n\n\n\n\n\n\n跳到主要'

In [7]:
# Extract the content after "入營前注意事項"
start_index = news_docs[0].page_content.find("入營前注意事項")

if start_index != -1:
    content_after_notice = news_docs[0].page_content[start_index:]

    # Extract the relevant portion for questions and answers
    qa_section = content_after_notice.split(" 新訓中心常見問題說明：")[1].split(" 十二、軍訓訓練課程折抵之文件要如何申請？")[0]

    # Add the twelfth question manually
    qa_section += " 十二、軍訓訓練課程折抵之文件要如何申請？ 答：可親自至原就讀學校申請成績單正本，送該校軍訓單位審查並計算得折減日數後加蓋印記，或查詢學校網站下載表格以通訊方式向學校申辦。"

    # Split the questions
    questions = [q.strip() for q in qa_section.split("。 ") if q]

    # Display the resulting list
    for idx, question in enumerate(questions):
        print(f"Question {idx + 1}: {question}")

    # Store in a list
    question_list = questions
else:
    print("The specified phrase '入營前注意事項' was not found in the document.")


Question 1: 一、入營後，新訓中心是否會提供衣物？ 答：入營報到後，由新訓中心提供迷彩服、運動服、短褲、鋼盔、鞋、襪、內衣褲及日常生活用品，如不敷使用，可依個人需求，至營區福利站自行購買
Question 2: 二、入營之役男，是否須先行理髮？ 答：役男入營前建議不要先行理髮，待入營後，由收訓營區統一安排，新兵入伍訓練髮式標準：採3mm(3分)，收費價廉且符合規定
Question 3: 三、新訓中心可否使用手機？ 答：可攜帶智慧型手機（中國大陸廠牌除外，禁止攝影、定位），入營後手機集中保管，定時定點開放使用，由於營區無法提供充電，可攜帶行動電源備用
Question 4: 四、新訓中心可否攜帶電動刮鬍刀？ 答：營區不提供充電，建議儘量使用手動刮鬍刀
Question 5: 五、新訓中心可否戴隱形眼鏡？ 答：於新兵訓練中心可配戴隱形眼鏡，但因新訓中心生活緊湊，而隱形眼鏡須相當時間消毒、清洗，以免感染影響視力，所以建議役男配戴鏡片眼鏡為宜，並可另行準備備份眼鏡，以免操課中毀損造成不便
Question 6: 六、在新訓中心，可否攜帶個人藥品？ 答：常備兵攜帶個人藥品入營，原則採集中專人保管，如須用藥可向專人領取服用；特殊疾病用藥者，可向單位先行報備後，於訓練時隨身攜帶（如：氣喘患者可攜帶氣管擴張劑）
Question 7: 七、在新訓中心身體不適如何處理？ 答：新兵於新訓中心身體不適，請隨時報告長官及時處理，各新訓中心均有醫務室，醫官會依病況作妥適之處理，如安排至醫務室休息或轉送軍醫院進一步檢查治療
Question 8: 八、在新訓中心之訓練要求方式為何？ 答：新訓中心的體能訓練原則是採循序漸進之方式，並會依新兵個人體能狀況分組訓練，讓新兵逐步適應跟上進度
Question 9: 九、在新訓中心之衣物洗滌方式？ 答：衣物之洗滌係自費採每日送洗方式，由入營新兵推選福利委員，對外招商辦理，新兵有個人洗衣袋，以免遺失或錯取
Question 10: 十、在新訓中心休假返家之交通？ 答：休假返家之交通方式，新兵可由家長接送返家或搭乘由新兵福利委員辦理之遊覽車原車自費往返
Question 11: 十一、在新訓中心是否有自動提款機可使用？ 答：目前新訓中心除少數營區(花蓮北埔及高雄左營)外均設有郵局自動提款機，以方便新兵提款
Question 12: 十二、軍訓訓練

In [8]:
question_list

['一、入營後，新訓中心是否會提供衣物？ 答：入營報到後，由新訓中心提供迷彩服、運動服、短褲、鋼盔、鞋、襪、內衣褲及日常生活用品，如不敷使用，可依個人需求，至營區福利站自行購買',
 '二、入營之役男，是否須先行理髮？ 答：役男入營前建議不要先行理髮，待入營後，由收訓營區統一安排，新兵入伍訓練髮式標準：採3mm(3分)，收費價廉且符合規定',
 '三、新訓中心可否使用手機？ 答：可攜帶智慧型手機（中國大陸廠牌除外，禁止攝影、定位），入營後手機集中保管，定時定點開放使用，由於營區無法提供充電，可攜帶行動電源備用',
 '四、新訓中心可否攜帶電動刮鬍刀？ 答：營區不提供充電，建議儘量使用手動刮鬍刀',
 '五、新訓中心可否戴隱形眼鏡？ 答：於新兵訓練中心可配戴隱形眼鏡，但因新訓中心生活緊湊，而隱形眼鏡須相當時間消毒、清洗，以免感染影響視力，所以建議役男配戴鏡片眼鏡為宜，並可另行準備備份眼鏡，以免操課中毀損造成不便',
 '六、在新訓中心，可否攜帶個人藥品？ 答：常備兵攜帶個人藥品入營，原則採集中專人保管，如須用藥可向專人領取服用；特殊疾病用藥者，可向單位先行報備後，於訓練時隨身攜帶（如：氣喘患者可攜帶氣管擴張劑）',
 '七、在新訓中心身體不適如何處理？ 答：新兵於新訓中心身體不適，請隨時報告長官及時處理，各新訓中心均有醫務室，醫官會依病況作妥適之處理，如安排至醫務室休息或轉送軍醫院進一步檢查治療',
 '八、在新訓中心之訓練要求方式為何？ 答：新訓中心的體能訓練原則是採循序漸進之方式，並會依新兵個人體能狀況分組訓練，讓新兵逐步適應跟上進度',
 '九、在新訓中心之衣物洗滌方式？ 答：衣物之洗滌係自費採每日送洗方式，由入營新兵推選福利委員，對外招商辦理，新兵有個人洗衣袋，以免遺失或錯取',
 '十、在新訓中心休假返家之交通？ 答：休假返家之交通方式，新兵可由家長接送返家或搭乘由新兵福利委員辦理之遊覽車原車自費往返',
 '十一、在新訓中心是否有自動提款機可使用？ 答：目前新訓中心除少數營區(花蓮北埔及高雄左營)外均設有郵局自動提款機，以方便新兵提款',
 '十二、軍訓訓練課程折抵之文件要如何申請？ 答：可親自至原就讀學校申請成績單正本，送該校軍訓單位審查並計算得折減日數後加蓋印記，或查詢學校網站下載表格以通訊方式向學校申辦。']

### QA問答

In [9]:
%%capture
!pip install python-dotenv
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install sentence_transformers
!pip install langchain_huggingface

In [10]:
import torch
from transformers import BitsAndBytesConfig

from langchain import LLMChain
from langchain.prompts import ChatPromptTemplate, PromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langchain.schema import StrOutputParser
from langchain_huggingface import ChatHuggingFace

from huggingface_hub import notebook_login

# chat model 需要使用 hf 的 token
notebook_login()

In [11]:
MODEL_NAME = "MediaTek-Research/Breeze-7B-Instruct-v0_1"

# 量化參數
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True)

# llm 初始化
llm = HuggingFacePipeline.from_model_id(
    model_id=MODEL_NAME,
    task="text-generation",
    model_kwargs=dict(
        torch_dtype=torch.float16,
        trust_remote_code=True,
        device_map="auto",
        quantization_config=quantization_config),
    pipeline_kwargs=dict(
        max_new_tokens=1024,
        temperature=0.0001,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.15,
        return_full_text=False) ) # 關閉完整回覆

# chat model 初始化
chat_llm = ChatHuggingFace(llm=llm)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
template = """

請你參考{source}中的資料來源回答以下問題。
問題: {question}, Let's think step by step.
答案:

"""

prompt = PromptTemplate(template=template, input_variables=["source","question"])

# 使用 LLM Chain 將 Prompt 與 LLM 串接起來
llm_chain = prompt | llm

# 將問題透過參數化的方式帶入
source = question_list
question = "入營後，新訓中心是否會提供衣物？"
print(llm_chain.invoke({"question": question, "source": source}))

入營報到後，由新訓中心提供迷彩服、運動服、短褲、鋼盔、鞋、襪、內衣褲及日常生活用品，如不敷使用，可依個人需求，至營區福利站自行購買。


## Advance
延伸 Basline 的內容，測試看看不同的 chunk 大小，對於生成結果的影響

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [25]:
for i in range(1,5):

  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=100*i,
      chunk_overlap=100)

  texts_chunks = text_splitter.split_documents(news_docs)

  prompt = PromptTemplate(template=template, input_variables=["source","question"])

  # 使用 LLM Chain 將 Prompt 與 LLM 串接起來
  llm_chain = prompt | llm

  # 將問題透過參數化的方式帶入
  source = question_list
  question = "我當兵吃東西犯法嗎"
  print('-------------\n',llm_chain.invoke({"question": question, "source": source}))

-------------
 1. 入營後，新訓中心是否會提供食物？
   - 答：入營報到後，由新訓中心提供三餐，包括早餐、午餐和晚餐。

2. 在新訓中心，可以帶零食來吃嗎？
   - 答：在新訓中心，可以自備零食，但需遵守食糧管理制度，避免影響健康與紀律。
-------------
 1. 入營後，新訓中心是否會提供食物？
   - 答：入營報到後，由新訓中心提供三餐，包括早餐、午餐和晚餐。

2. 在新訓中心，可以帶零食來吃嗎？
   - 答：在新訓中心，可以自備零食，但需遵守食糧管理制度，避免影響健康與紀律。
-------------
 1. 入營後，新訓中心是否會提供食物？
   - 答：入營報到後，由新訓中心提供三餐，包括早餐、午餐和晚餐。

2. 在新訓中心，可以帶零食來吃嗎？
   - 答：在新訓中心，可以自備零食，但需遵守食糧管理制度，避免影響健康與紀律。


KeyboardInterrupt: 